In [1]:
import pandas as pd
from io import StringIO
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

from openai import OpenAI
atwin_api=#censored for now

oai_client=OpenAI(api_key=atwin_api)

In [37]:
def calculate_amount(group):
    if group["ML_ID"].str.startswith('cycle').any():
        return group["TX_AMOUNT"].max()
    elif group["ML_ID"].str.startswith('fan').any():
        return group["TX_AMOUNT"].sum()
    elif group["ML_ID"].str.startswith(('gather', 'scatter')).any():
        return group["TX_AMOUNT"].sum() / 2
    else:
        return 0
    
    
def create_summary(row, final_relationship):
    ml_cases_cols=['n_entities_involved','total_amount'] #n entities involved et to be generated in the ml_cassese generation function
    
    ml_id = row['ML_ID']
    ml_type = row['ML_type']
    
    def retrieve_relevant_dfs(final_relationship, ml_id):
        final_relationship_cols = ['RECEIVER_ACCOUNT', 'SENDER_ACCOUNT', 'TX_TYPE', 'TX_AMOUNT', 'TIMESTAMP']
        return final_relationship[final_relationship['ML_ID'] == ml_id][final_relationship_cols]
    def summarize(system_prompt,prompt,model='gpt-4o'):
        completion = oai_client.chat.completions.create(
            model=model,
            temperature=0,
            messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content
    
    
    relevant_final_rel = retrieve_relevant_dfs(final_relationship, ml_id)
    if relevant_final_rel.empty:
        return "No relevant final relationship data found."
    
    relevant_ml_cases_info=row[ml_cases_cols]
    context = f'''Complete trx records of the suspected {ml_type}:
{relevant_final_rel}
Trx summary:
{relevant_ml_cases_info}'''

    explainable_system_prompt='''
        Write a summary explaining what type of mony laundering and describe in points as to why is the given trx scheme a suspicious money laundering case. 
        Keep it short, sweet, and easy to understand for fraud analysts of a financial institution.
        Use emphasis with **bold** for emphasis on names and type of money laundering (smurf or round-trip). DOn't use title or header, just go straight into the explanation.
    '''    
    prompt=f'''Tulis rangkuman singkat (di bawah 200 kata) dalam Bahasa Indonesia untuk skema transaksi berikut\n{context}'''
    summary=summarize(explainable_system_prompt,prompt)
    return summary

# Function to get the first transaction date
def get_first_trx_date(group):
    return group["TIMESTAMP"].min()

# def get_key_entities(group):
#     entities_amt_sentreceived = pd.concat([group[['RECEIVER_ACCOUNT', 'TX_AMOUNT']], group[['SENDER_ACCOUNT', 'TX_AMOUNT']].rename(columns={'SENDER_ACCOUNT': 'RECEIVER_ACCOUNT'})])
#     top_entities = entities_amt_sentreceived.groupby('RECEIVER_ACCOUNT')['TX_AMOUNT'].sum().nlargest(3).index.tolist()
#     sep=';'
#     return f'{sep} '.join(top_entities)

def get_key_entities(group):
    # Combine the receiver and sender accounts into a single DataFrame
    entities_amt_sentreceived = pd.concat([
        group[['RECEIVER_ACCOUNT', 'TX_AMOUNT', 'TX_TYPE']],
        group[['SENDER_ACCOUNT', 'TX_AMOUNT', 'TX_TYPE']].rename(columns={'SENDER_ACCOUNT': 'RECEIVER_ACCOUNT'})
    ])

    # Filter out rows with TX_TYPE in ('wire', 'trade', 'loan') and TX_AMOUNT equal to 0
    filtered_entities = entities_amt_sentreceived[
        (entities_amt_sentreceived['TX_AMOUNT'] == 0) & 
        (~entities_amt_sentreceived['TX_TYPE'].isin(['wire', 'trade', 'loan']))
    ]
    zero_tx_entities = filtered_entities['RECEIVER_ACCOUNT'].unique().tolist()
    grouped_entities = entities_amt_sentreceived.groupby('RECEIVER_ACCOUNT')['TX_AMOUNT'].sum()
    top_entities = grouped_entities[grouped_entities > 0].nlargest(3).index.tolist()

    all_entities = zero_tx_entities + top_entities

    sep = '; '
    return f'{sep} '.join(all_entities)

def count_entities(group):
    unique_entities = pd.concat([group['RECEIVER_ACCOUNT'], group['SENDER_ACCOUNT']]).unique()
    return len(unique_entities)

def determine_ml_type(group):
    # Extract the string before the first space in the 'ML_ID' column
    return group['ML_ID'].iloc[0].split('_')[0]

In [39]:
final_relationship=pd.read_csv('final_relationship_finale.csv')
ml_cases = final_relationship.groupby("ML_ID").apply(lambda group: pd.Series({
    "ML_type": determine_ml_type(group),
    "first_trx_date": get_first_trx_date(group),
    "total_amount": calculate_amount(group),
    "key_entities": get_key_entities(group),
    "n_entities_involved": count_entities(group)
})).reset_index()
ml_cases['explaination'] = ml_cases.apply(create_summary, axis=1, final_relationship=final_relationship)

C:\Users\BNI\AppData\Local\Temp\ipykernel_15112\1016181748.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ml_cases = final_relationship.groupby("ML_ID").apply(lambda group: pd.Series({


In [41]:
ml_cases.to_csv('ml_cases_finale2.csv',index=0)

In [4]:
final_relationship=pd.read_csv('final_relationship_finale.csv')


In [6]:
final_relationship.to_csv('final_relationship_finale.csv',index=0)

In [45]:
final_relationship[final_relationship['ML_ID']=='fan-out_3']

,Unnamed: 0,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE
105,14,Victor King,0037048,RM,22184.0,2.900000e+04,2022-08-01 14:46:00,fan-out_3,wire
106,15,Walter Scott,Victor King,IDR,122407.0,8.591281e+07,2022-08-06 12:08:00,fan-out_3,wire
107,16,Xander Young,Victor King,IDR,15398.0,8.055719e+07,2022-08-11 21:23:00,fan-out_3,wire
108,17,Yvonne Hall,Victor King,USD,4252.0,5.143776e+04,2022-08-14 16:10:00,fan-out_3,wire
109,18,Zach Lee,Victor King,RM,1512.0,2.900000e+04,2022-08-18 12:06:00,fan-out_3,wire
110,19,Quinn Perez,Victor King,IDR,8917.0,9.222559e+07,2022-08-20 11:14:00,fan-out_3,wire
111,20,Ralph Brown,Victor King,SGD,50719.0,5.729732e+04,2022-08-22 12:44:00,fan-out_3,wire
112,21,Steve Clark,Victor King,IDR,5610.0,9.746290e+07,2022-08-24 09:28:00,fan-out_3,wire
113,22,Tiffany Adams,Victor King,IDR,60001.0,8.922664e+07,2022-08-31 11:04:00,fan-out_3,wire
114,23,Uma Turner,Victor King,RM,13305.0,2.900000e+04,2022-09-02 18:55:00,fan-out_3,wire


In [40]:
ml_cases

,ML_ID,ML_type,first_trx_date,total_amount,key_entities,n_entities_involved,explaination
0,cycle_1,cycle,2000-01-01 00:00:00,1.500000e+07,Masa Metonia Abadi; Musmuliadi; Citra Lestar...,8,**Jenis Pencucian Uang: Round-Trip**\n\n**Alas...
1,cycle_2,cycle,2022-08-01 14:49:00,6.113822e+04,Bob Smith; Judy Harris; Grace Taylor,10,**Jenis Pencucian Uang: Round-Trip**\n\nSkema ...
2,cycle_3,cycle,2022-08-01 14:55:00,1.684000e+03,Oscar White; Mallory Martin; Trent Moore,3,**Tipe Pencucian Uang: Round-Trip**\n\n**Alasa...
3,cycle_4,cycle,2022-08-01 00:19:00,1.378737e+06,John Doe; Daniel Thompson; Jane Smith,12,**Jenis pencucian uang:** **Round-trip**\n\n**...
4,cycle_6,cycle,2022-08-01 02:58:00,1.157274e+05,Hari Nugroho; Indah Rahayu; Ahmad Sutrisno,9,Skema transaksi ini menunjukkan pola **money l...
5,cycle_7,cycle,2022-08-01 07:35:00,6.043311e+04,Utari Puspita; Vera Melati; Yani Wulandari,12,**Jenis Pencucian Uang: Round-Trip**\n\nSkema ...
6,cycle_8,cycle,2000-01-01 00:00:00,9.572920e+04,Linggar Jati Perkasa; Liris Enggar Wardhani; ...,13,Jenis pencucian uang yang teridentifikasi dala...
7,fan-in_1,fan-in,2022-08-01 00:08:00,6.750000e+06,Olivia Harris; Olivia Adams; Olivia Baker,10,**Fan-in** adalah jenis pencucian uang di mana...
8,fan-out_1,fan-out,2022-08-01 12:46:00,8.920212e+08,Boni Arfan; Ferdi Santoso; Bambang Suprapto,15,**Jenis Pencucian Uang: Smurfing**\n\nSkema tr...
9,fan-out_2,fan-out,2022-08-01 13:24:00,8.434489e+07,Indra Wijaya; Lina Marlina; Doni Nugroho,13,**Fan-out** adalah jenis pencucian uang di man...


In [34]:
ml_cases

,ML_ID,ML_type,first_trx_date,total_amount,key_entities,n_entities_involved
0,cycle_1,cycle,2000-01-01 00:00:00,1.500000e+07,Masa Metonia Abadi; Musmuliadi; Citra Lestar...,8
1,cycle_2,cycle,2022-08-01 14:49:00,6.113822e+04,Bob Smith; Judy Harris; Grace Taylor,10
2,cycle_3,cycle,2022-08-01 14:55:00,1.684000e+03,Oscar White; Mallory Martin; Trent Moore,3
3,cycle_4,cycle,2022-08-01 00:19:00,1.378737e+06,John Doe; Daniel Thompson; Jane Smith,12
4,cycle_6,cycle,2022-08-01 02:58:00,1.157274e+05,Hari Nugroho; Indah Rahayu; Ahmad Sutrisno,9
5,cycle_7,cycle,2022-08-01 07:35:00,6.043311e+04,Utari Puspita; Vera Melati; Yani Wulandari,12
6,cycle_8,cycle,2000-01-01 00:00:00,9.572920e+04,Linggar Jati Perkasa; Liris Enggar Wardhani; ...,13
7,fan-in_1,fan-in,2022-08-01 00:08:00,6.750000e+06,Olivia Harris; Olivia Adams; Olivia Baker,10
8,fan-out_1,fan-out,2022-08-01 12:46:00,8.920212e+08,Boni Arfan; Ferdi Santoso; Bambang Suprapto,15
9,fan-out_2,fan-out,2022-08-01 13:24:00,8.434489e+07,Indra Wijaya; Lina Marlina; Doni Nugroho,13


In [28]:
final_relationship['ML_ID']=final_relationship['ML_ID'].str.replace(' ','_')

In [10]:
# Group by ML_ID and apply the functions
ml_cases = final_relationship.groupby("ML_ID").apply(lambda group: pd.Series({
    "ML_type": determine_ml_type(group),
    "first_trx_date": get_first_trx_date(group),
    "total_amount": calculate_amount(group),
    "key_entities": get_key_entities(group),
    "n_entities_involved": count_entities(group)
})).reset_index()


KeyError: 'SENDER_ACCOUNT'

In [11]:
ml_cases

,ML_ID,ML_type,first_trx_date,total_amount,key_entities,n_entities_involved
0,cycle 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER...,2000-01-01 00:00:00,1.500000e+07,Citra Lestari; Masa Metonia Abadi; Budi Seti...,8
1,cycle 2,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 14:49:00,6.113822e+04,Bob Smith; Judy Harris; Grace Taylor,10
2,cycle 3,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 14:55:00,1.684000e+03,Oscar White; Mallory Martin; Trent Moore,3
3,cycle 4,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 00:19:00,1.378737e+06,John Doe; Daniel Thompson; Jane Smith,12
4,cycle 6,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 02:58:00,1.157274e+05,Hari Nugroho; Indah Rahayu; Ahmad Sutrisno,9
5,cycle 7,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 07:35:00,6.043311e+04,Utari Puspita; Vera Melati; Yani Wulandari,12
6,cycle 8,Unnamed: 0 RECEIVER_ACCOUNT ...,2000-01-01 00:00:00,9.572920e+04,Dadan Saputra; Cici Kartika; Kiki Sumirat,13
7,fan-in 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOUN...,2022-08-01 00:08:00,6.750000e+06,Olivia Harris; Olivia Adams; Olivia Baker,10
8,fan-out 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 12:46:00,8.920212e+08,Boni Arfan; Ferdi Santoso; Bambang Suprapto,15
9,fan-out 2,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOUNT...,2022-08-01 13:24:00,8.434489e+07,Indra Wijaya; Lina Marlina; Doni Nugroho,13


In [8]:
ml_cases

,ML_ID,ML_type,first_trx_date,total_amount,key_entities,n_entities_involved
0,cycle 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER...,2000-01-01 00:00:00,1.500000e+07,Citra Lestari; Masa Metonia Abadi; Budi Seti...,8
1,cycle 2,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 14:49:00,6.113822e+04,Bob Smith; Judy Harris; Grace Taylor,10
2,cycle 3,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 14:55:00,1.684000e+03,Oscar White; Mallory Martin; Trent Moore,3
3,cycle 4,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 00:19:00,1.378737e+06,John Doe; Daniel Thompson; Jane Smith,12
4,cycle 6,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOU...,2022-08-01 02:58:00,1.157274e+05,Hari Nugroho; Indah Rahayu; Ahmad Sutrisno,9
5,cycle 7,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 07:35:00,6.043311e+04,Utari Puspita; Vera Melati; Yani Wulandari,12
6,cycle 8,Unnamed: 0 RECEIVER_ACCOUNT ...,2000-01-01 00:00:00,9.572920e+04,Dadan Saputra; Cici Kartika; Kiki Sumirat,13
7,fan-in 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOUN...,2022-08-01 00:08:00,6.750000e+06,Olivia Harris; Olivia Adams; Olivia Baker,10
8,fan-out 1,Unnamed: 0 RECEIVER_ACCOUNT SENDER_AC...,2022-08-01 12:46:00,8.920212e+08,Boni Arfan; Ferdi Santoso; Bambang Suprapto,15
9,fan-out 2,Unnamed: 0 RECEIVER_ACCOUNT SENDER_ACCOUNT...,2022-08-01 13:24:00,8.434489e+07,Indra Wijaya; Lina Marlina; Doni Nugroho,13
